# Browser Automation Homework ROUND 2
Due 7-16<br>
Completed by: **TK YOUR NAME**

We're going to the United States trademark data base and going to collect all of Nike (or any company's) active trademarks.

We only want the live trademarks, and we want the `serial` number and the link to an image of each trademark. Feel free to collect the `wordmark` if you like.


Note: if you get asked if you're a bot, just complete the challenges manually.

In [80]:
import os
import random
import time

from playwright.async_api import async_playwright, expect, Keyboard

In [81]:
os.makedirs('data/', exist_ok=True)

In [82]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [83]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.firefox.launch(headless=headless)

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [84]:
driver, page = await open_browser()

In [85]:
url = 'https://tmsearch.uspto.gov/search/'
await page.goto(url)

<Response url='https://tmsearch.uspto.gov/search/' request=<Request url='https://tmsearch.uspto.gov/search/' method='GET'>>

Only search by the "Owner" of each trademark

In [86]:
xpath_search_options = '//mat-select[@aria-label="Search refinment"]'
filter = page.locator(xpath_search_options)

In [87]:
await filter.click()

In [88]:
xpath_option = '//mat-option[@id="mat-option-11"]'

In [89]:
await page.locator(xpath_option).click()

Find the search bar and input the search

In [90]:
xpath_searchbar = '//input[@aria-label="Search trademarks"]'

In [91]:
search = page.locator(xpath_searchbar)

In [92]:
# Change this if you like
company = 'Nike'

In [96]:
await search.fill(company)

In [97]:
await page.keyboard.press('Enter')

Filter to only live contracts
<br>In other words, filter out the dead ones.

In [98]:
xpath_filter = '//input[@id="statusDead"]'

In [99]:
dead_button = page.locator(xpath_filter)

In [100]:
await dead_button.click()

# Save results

In [101]:
# how to save what the emulator sees
source = await page.content()
with open(f'data/trademarks_{company}.html', 'w') as f:
    f.write(source)

GO TO THE NEXT PAGE AND REPEAT

In [102]:
xpath_next = '//li[@class="page-item"]//i[text() = "navigate_next"]'

In [103]:
xpath_result = '//div[@id="resultsRow"]'

In [105]:
next_button = page.locator(xpath_next)

In [106]:
await expect(next_button).to_be_visible()

In [107]:
await next_button.click()

In [25]:
await page.locator(xpath_result).is_visible()

True

In [31]:
page.locator(xpath_next)

<Locator frame=<Frame name= url='https://tmsearch.uspto.gov/search/search-results'> selector='//li[@class="page-item"]//i[text() = "navigate_next"]'>

In [32]:
await expect(page.locator(xpath_next)).to_be_visible()

In [108]:
i = 1
collect = True
while collect:
    try: # note changes in the try and `expect` syntax
        # page.is_visible(xpath_next)
        # check the next button is visible
        await expect(page.locator(xpath_next)).to_be_visible(timeout=1000)
        
        # click the next button
        next_button = page.locator(xpath_next)
        await next_button.click()
    
        # make sure the search result is visible
        await page.locator(xpath_result).is_visible()
    
        # save the contents
        source = await page.content()
        with open(f'data/trademarks_{company}_{i}.html', 'w') as f:
            f.write(source)
        i += 1

        if page.locator(xpath_end).is_visible():
            break
    except Exception as e:
        collect = False
        break

## Parse the contents

In [36]:
import glob
import pandas as pd
from lxml import etree, html

In [37]:
files = glob.glob('data/trademarks*.html')

In [38]:
xpath_trademark_cards = './/div[@class="card m-2 result-card ng-star-inserted"]'
xpath_serial = './/div[@class="row mb-2 ng-star-inserted"]//span'

In [75]:
data = []
for fn in files:
    # read into a XML tree from a string
    tree = html.fromstring(open(fn).read())
    trademarks = tree.findall(xpath_trademark_cards)
    for trademark in trademarks:
        serial = trademark.find(xpath_serial)

        # note this change here:
        img_url = trademark.findall('.//img')
        if img_url:
            img_url = img_url[0].get('src')
        else:
            img_url = None
            
        row = {'serial' : serial.text, 'img_url': img_url}
        data.append(row)

In [76]:
df = pd.DataFrame(data)
df.head(5)

,serial,img_url
0,73361065,https://tmcms-docs.uspto.gov/cases/73361065/ma...
1,73714272,None
2,74132298,None
3,73190217,None
4,73728115,https://tmcms-docs.uspto.gov/cases/73728115/ma...


In [77]:
df.to_csv('data/trademarks.csv', index=False)

In [79]:
await driver.close()

Open Data

Official API

Browser Automation

Undocumented APIs 

Static HTML requsts

Crowdsourcing / FOIA